# Compression de Lempel-Ziv-Welch (LZW)

LZW détermine un codage (dans un dictionnaire d) au fur et à mesure de la lecture du texte. On va coder certains motifs.

Intéressant quand les motifs se répètent.

**compression de LZW**

Initilament, les n lettres sont codées par un entier.  
Tant qu'il reste du texte $s$ à coder:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Retirer le plus long préfixe $w$ de $s$ qui soit dans $d$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Afficher le codage de $w$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$w'$ <- $w$ concaténé avec la prochaine lettre de $s$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ajouter un nouveau codage pour $w'$ dans $d$

In [7]:
let code s =
  let n = String.length s in
  let d = Hashtbl.create 13 in
  let dinv = Hashtbl.create 13 in
  let max_code = ref 0 in
  let add_code k v =
      Hashtbl.add d k v;
      Hashtbl.add dinv v k in
  for i = 0 to n - 1 do (* initialize d *)
    if not (Hashtbl.mem d (String.sub s i 1)) then (
      add_code (String.sub s i 1) !max_code;
      incr max_code
    )
  done;
  let rec aux i = (* returns compression of s[i:] *)
    if i >= String.length s then []
    else
      let rec max_suffix j =
        if j = n || not (Hashtbl.mem d (String.sub s i (j - i + 1))) then j
        else max_suffix (j + 1) in
      let j = max_suffix i in
      let w = String.sub s i (j - i) in
      if j < n then (
        let w' = String.sub s i (j - i + 1) in
        add_code w' (!max_code + 1);
        incr max_code;
      );
      (Hashtbl.find d w)::aux j
  in
  aux 0, dinv

val code : string -> int list * (int, string) Hashtbl.t = <fun>

In [9]:
let c, dinv = code "barbapapa"

val c : int list = [0; 1; 2; 5; 3; 1; 9]
val dinv : (int, string) Hashtbl.t = <abstr>

**décompression de LZW**

* Pour décoder, on a besoin de stocker le dictionnaire réciproque de $d$
* On peut imposer une longeur maximum d'un motif codé

In [14]:
let rec lzw_decode = function
    | [] -> ""
    | e::q -> (Hashtbl.find dinv e)^(lzw_decode q)

val lzw_decode : int list -> string = <fun>